In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/DontGetKicked/training.csv')
test = pd.read_csv('/kaggle/input/DontGetKicked/test.csv')
sub = pd.read_csv('/kaggle/input/DontGetKicked/example_entry.csv')

In [ ]:
alldata = pd.concat([train,test])
alldata

In [ ]:
alldata['PurchDate'] = pd.to_datetime(alldata['PurchDate'])
alldata['year'] = alldata['PurchDate'].dt.year
alldata['month'] = alldata['PurchDate'].dt.month
alldata['day'] = alldata['PurchDate'].dt.day

In [ ]:
alldata2 = alldata.drop(columns=['IsBadBuy','RefId','PurchDate'])
alldata2

In [ ]:
alldata2['Auction'].unique()

In [ ]:
alldata2['Auction'] = alldata2['Auction'].replace({'ADESA':0,'OTHER':1,'MANHEIM':2})
alldata2

In [ ]:
alldata2['Make'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
alldata2['Make'] = le.fit_transform(alldata2['Make'])
alldata2

In [ ]:
c = alldata2.columns[alldata2.dtypes== object]#list는 조건을 쓸 수 있다!  문자형칼럼에 접근 object는 따옴표있어도 괜춘없어도 상관없음
c#전체 데이터에 접근한다. 

In [ ]:
for i in c:
    alldata2[i] = le.fit_transform(alldata2[i]) #i에 따옴표 필요없엉
alldata2 
#alldata2를 for문에 돌려야해요. train과 test를 따로 하면은 레이블이 다르게 적용될 수 있어요!


In [ ]:
pd.options.display.max_rows=999
alldata2.head(20)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,10))
sns.boxplot(alldata['IsBadBuy'],alldata['WheelType'])
#boxplot은 수치와 카테고리
#countplot 둘다 카테고리
#scatterplot은 둘다 수치

In [ ]:
alldata['Model'].unique()

In [ ]:
alldata2['Model'] = le.fit_transform(alldata2['Model'])
alldata2['Trim'] = le.fit_transform(alldata2['Trim'])
alldata2['SubModel'] = le.fit_transform(alldata2['SubModel'])
alldata2['Color'] = le.fit_transform(alldata2['Color'])
alldata2['Transmission'] = le.fit_transform(alldata2['Transmission'])

In [ ]:
alldata2['WheelType'] = le.fit_transform(alldata2['WheelType'])
alldata2['VehOdo'] = le.fit_transform(alldata2['VehOdo'])
alldata2['Nationality'] = le.fit_transform(alldata2['Size'])
alldata2['TopThreeAmericanName'] = le.fit_transform(alldata2['TopThreeAmericanName'])
alldata2['Size'] = le.fit_transform(alldata2['Size'])
alldata2['PRIMEUNIT'] = le.fit_transform(alldata2['PRIMEUNIT'])
alldata2['AUCGUART'] = le.fit_transform(alldata2['AUCGUART'])
alldata2['VNST'] = le.fit_transform(alldata2['VNST'])


In [ ]:
pd.options.display.max_columns=999
alldata2

In [ ]:
alldata2 = alldata2.fillna(-1) #boost은 결측치가 있어도 돌아갑니다! 하지만 random은 불가해요!!!

In [ ]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1) #-1은 최대값을 쓴다. 최종 많은거는 4개이다. #데이터가 많으면 시간이 걸려요. 10만개정도면 많으니깐 cpu를 다 쓰지 않아요. 캐글은 최대 4개
rfc.fit(train2,train['IsBadBuy'])
result = rfc.predict_proba(test2) #분류는 꼭!! predict_proba로 하세요!

In [ ]:
result[:50]
#칼럼이 2개인 경우가 0과 1이 나오는 거니깐!! 왼쪽은 0 사기가 아닌 확률 , 오른쪽 1은 사기를 당할 확률 
#보통은 1이 중요한거니깐 오른쪽을 제출합니다!
#이대로 넣으면 자동으로 왼쪽이 나와요! 그러니 반드시 오른쪽에 접근하는 코드를 배웁시다!!

In [ ]:
#분류를 할 때 제대로 하지 않아서 입니다. 
#기존에는 regression문제였다. 
#분류에 있어서 너무 이분법적으로 모델이 판단합니다. 
#현재는 정도가 없습니다. 대출을 잘 갚는 사람중에서도 아슬아슬한 사람과...1%만큼 잘 갚는 사람이 있을 것입니다. 퍼센테이지로 알아낼 수 있습니다. 

#충분히 잘 갚는지 아닌지...퍼센테이지 49% 51%는 잘 낼 수도 있는 사람..일지도..
#거의 대부분의 분류문제는 확률값으로 나가게 해야합니다!
#제출할 때 꼭 확률로 제출하지 않아도 됩니다. 
#95%의 대회는 확률로 나옵니다!ㅎ


In [ ]:
test

In [ ]:
pd.Series(rfc.feature_importances_,index=train2.columns).sort_values(ascending=False)

In [ ]:
sub['IsBadBuy'] = result[:,-1]#조건을 주는 대괄호 쉼표를 기준으로 왼쪽은 row 오른쪽이 colm
#현재는 이진분류, 나중에는 다중분류!를 배우도록 하겠습니다~!ㅎ
sub

In [ ]:
sub.to_csv('dont_수업이다유.csv',index=0)

In [ ]:
#label encoder 와 fillna-1로 채우니깐 0.10336점으로 477등이 나옴
#날짜 칼럼을 추가하니깐 0.10347점으로 점수가 더 좋아짐


#날짜 칼럼을 추가해보았습니다. #하면서 궁금한 점은 트리형모델들은 학습에 좋지 않는 feature를 지닌 칼럼은 자동으로 딴 주머니로 넣어주니
#일단은 칼럼을 많이 만들어 놓는게 좋은걸까요?? 그렇다.!!!!  몇백개 천개미만은 가능합니다!!
#중요한 칼럼을 찾을 때는 어떻게 접근을 해야하는지 모르겠습니다. 
#le.fit_transform(alldata2['label'])이거를 한번에 해결하는 for문 코드 알려주세요. 



###모델이 확인했을 때는 칼럼에서 카테고리가 많으면 더 중요하다고 생각한다...ㅎ수치데이터는 그렇게 판단하지는 않습니다!!! 이상하네~~!
##알아서 숫자형 칼럼과 카테고리적인 느낌의 칼럼을 구분할 수 있습니다. 


In [ ]:
#roc,auc곡선은 나름대로 우리의 모델이 0.4라고 예측했다면.. 
#roc곡선! 지니계수를 찾아보기
# roc 0.5까지이니..구분이 좀 덜 됩니다. 
#지니계수는 0~1이기에 구분이 좀 더 잘 됩니다!